In [1]:
# 
# Module Zeno
# RLH 1/29/2021. updated 11/26/21. Added Lt, step transmission length 12/4/21.
#
module zeno

using ProgressBars

@doc """
    zenogrowth(Lmax, Hmax, tmax; F=0.001, a=1.0, Ls=1.0, Lc=2.0, Lt=0.0, Ld=1.0, B=0.0)

Minimal model for growth in 1+1 dimensions.

Example: ```zenogrowth(200,400,200000)```
Create a mound with a fixed base layer 200 units long, with a maximum height of 400, calculated in 200k time steps. 
Return Hbot, Htop, L_array, TerraceLength, v.
"""
function zenogrowth(Lmax, Hmax, tmax; F=0.001, a=1.0, Ls=1.0, Lc=2.0, Ld=1.0, B=0.0)   
    rms_array = zeros(tmax)
    avg_h_array = zeros(tmax)
    
    Hbot = 1
    Htop = 2
    sticking(L) = 1.0-B*((L^2)/(L+Ld)^2) # L-dependent sticking coefficient
    f(L,Ls,isTop) = (isTop ? (a*F*L)*sticking(L) : (a*F*L/2)*sticking(L) * (1 - Ls/(Ls+L))) # from upper terrace
    g(L,Ls,isBot) = (isBot ? (a*F*L)*sticking(L) : (a*F*L/2)*sticking(L) * (1 + Ls/(Ls+L))) # from lower terrace

    v = zeros(Hmax)
    L_array = zeros(Hmax)
    TerraceLength = zeros(Hmax)
    L_array[1] = Lmax


    for i in tqdm(1:tmax,printing_delay=60)
        L_0 = copy(L_array)

        # Calculate the terrace lengths.
        TerraceLength = zeros(Hmax)
        for j in Hbot:Htop
            TerraceLength[j] = L_array[j] - L_array[j+1]
        end

        newHbot = 1
        for j in Hbot:Htop
            if L_array[j] >= Lmax
                newHbot = j
            end
        end
        Hbot = newHbot
    
        Htop_temp = copy(Htop)
        Hbot_temp = copy(Hbot)    
    
        # Calculate the step velocities.
        # Changed Htop to Htop-1 to avoid nucleating early. RLH 11/25/21
        v = zeros(Hmax)
        for j in Hbot:Htop-1
            if L_array[j] >= Lmax      
                if L_array[j+1] < Lc
                    v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)       
                else
                    v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                end
                
            elseif L_array[j+1] < Lc
                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
            else
                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
            end
        end
    
        trigger_Lmax = 0 # trigger for Lmax correction
       
        for j in Hbot:Htop
            if L_array[j] >= Lc
                L_array[j+1] += v[j]
                    if L_array[j+1] >= Lmax                   
                        trigger_Lmax = 1 # Lmax correction happened
                    end
            end
        end
    
        trigger_Lc = 0 # trigger for Lc correction
    
        if L_array[Htop] > Lc
            trigger_Lc = 1 # Lc correction needed
        end
        
        # update Htop
        for j in Hbot:Htop
            if L_array[j] >= Lc
                if j==Htop 
                    Htop+=1            
                end
            end
        end
        
        # only Lmax correction is needed, Lc correction is not 
        if trigger_Lmax == 1
            if trigger_Lc == 0
        
                L_step_1 = copy(L_0)  # return to previous step positiion
        
                # return to the old Hot and Hbot
                Htop = copy(Htop_temp)
                Hbot = copy(Hbot_temp)
        
                # Calculate the terrace lengths.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_1[j] - L_step_1[j+1]
                end
                
                # find length needed to reach exact Lmax
                t_length = TerraceLength[Hbot]

                #flux for sub step 1
                flux_1 =  t_length/(TerraceLength[Hbot]*sticking(TerraceLength[Hbot]) + TerraceLength[Hbot+1]/2*sticking(TerraceLength[Hbot+1])*(1-Ls/(Ls+TerraceLength[Hbot+1])))
   
        
                F_temp = copy(F) # save the default flux
                F = copy(flux_1) # replace the flux with flux_1 for sub step 1

        
                # recalculate the v_array for sub step 1
                v = zeros(Hmax)
                for j in Hbot:Htop-1
                    if L_step_1[j] >= Lmax      
                        if L_step_1[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                        end
            
                    elseif L_step_1[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                    end
                end
                            
                #update the L array with sub step 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lc
                        L_step_1[j+1] += v[j]
                    end
                end
            
                # find correct Htop after sub step 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lc
                        if j==Htop 
                            Htop+=1            
                        end
                    end
                end
        
        
                # find correct Hbot for sub step 2
                newHbot = 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lmax
                        newHbot = j
                    end
                end
                Hbot = newHbot
        
                L_step_2 = copy(L_step_1)
        
                flux_2 = F_temp - flux_1
        
                # Calculate the terrace lengths again for sub step 2.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_2[j] - L_step_2[j+1]
                end
                    
                F = copy(flux_2) # replace the flux with flux_2 for sub step 2
        
                # recalculate the v_array for sub step 2
                v = zeros(Hmax)
                for j in Hbot:Htop-1
                    if L_step_2[j] >= Lmax      
                        if L_step_2[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                        end
            
                    elseif L_step_2[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                    end
                end
                
                #update the L array with sub step 2
                for j in Hbot:Htop
                    if L_step_2[j] >= Lc
                        L_step_2[j+1] += v[j]
                    end
                end
            
                # find correct Htop after sub step 2
                for j in Hbot:Htop
                    if L_step_2[j] >= Lc
                        if j==Htop 
                            Htop+=1            
                        end
                    end
                end

                  
                F = copy(F_temp) # return the flux to the default
                L_array = copy(L_step_2)
            end
        end
    
        # only Lc correction is needed, Lmax correction is not 
        if trigger_Lc == 1
            if trigger_Lmax == 0
            
                L_step_1 = copy(L_0)  # return to previous step positiion
        
                # return to the old Hot and Hbot
                Htop = copy(Htop_temp)
                Hbot = copy(Hbot_temp)
        
                # Calculate the terrace lengths.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_1[j] - L_step_1[j+1]
                end
                
                # find length needed to reach exact Lc
                t_length = Lc - TerraceLength[Htop]

            
                #flux for sub step 1
                flux_1 =  t_length/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]/2*sticking(TerraceLength[Htop-1])*(1+Ls/(Ls+TerraceLength[Htop-1])))
                # special case for layer 1 and 2
                if Hbot == 1
                    if Htop == 2
                        flux_1 = t_length/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]*sticking(TerraceLength[Htop-1]))
                    end
                end
        
                F_temp = copy(F) # save the default flux
                F = copy(flux_1) # replace the flux with flux_1 for sub step 1

        
                # recalculate the v_array for sub step 1
                v = zeros(Hmax)
                for j in Hbot:Htop-1
                    if L_step_1[j] >= Lmax      
                        if L_step_1[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                        end
            
                    elseif L_step_1[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                    end
                end
            
                    
                #update the L array with sub step 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lc
                        L_step_1[j+1] += v[j]
                    end
                end
                        
                # find correct Htop after sub step 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lc
                        if j==Htop 
                            Htop+=1            
                        end
                    end
                end
        
        
                # find correct Hbot for sub step 2
                newHbot = 1
                for j in Hbot:Htop
                    if L_step_1[j] >= Lmax
                        newHbot = j
                    end
                end
                Hbot = newHbot
        
                L_step_2 = copy(L_step_1)
                flux_2 = F_temp - flux_1
        
                # Calculate the terrace lengths again for sub step 2.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_2[j] - L_step_2[j+1]
                end
                    
                F = copy(flux_2) # replace the flux with flux_2 for sub step 2
        
                # recalculate the v_array for sub step 2
                v = zeros(Hmax)
                for j in Hbot:Htop-1
                    if L_step_2[j] >= Lmax      
                        if L_step_2[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                        end
            
                    elseif L_step_2[j+1] < Lc
                        v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                    else
                        v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                    end
                end

                #update the L array with sub step 2
                for j in Hbot:Htop
                    if L_step_2[j] >= Lc
                        L_step_2[j+1] += v[j]
                    end
                end
                        
                # find correct Htop after sub step 2
                for j in Hbot:Htop
                    if L_step_2[j] >= Lc
                        if j==Htop 
                            Htop+=1            
                        end
                    end
                end

                F = copy(F_temp) # return the flux to the default
                L_array = copy(L_step_2)    
            end
        end
    
    
        # Both Lc and Lmax correction is needed
        if trigger_Lmax == 1
            if trigger_Lc == 1
            
                 L_step_1 = copy(L_0)  # return to previous step positiion
        
                # return to the old Hot and Hbot
                Htop = copy(Htop_temp)
                Hbot = copy(Hbot_temp)
        
                # Calculate the terrace lengths.
                TerraceLength = zeros(Hmax)
                for j in Hbot:Htop
                    TerraceLength[j] = L_step_1[j] - L_step_1[j+1]
                end
                
                # find length needed to reach exact Lc
                t_length_Lc = Lc - TerraceLength[Htop]
            
                #flux for sub step 1
                flux_1_Lc =  t_length_Lc/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]/2*sticking(TerraceLength[Htop-1])*(1+Ls/(Ls+TerraceLength[Htop-1])))
                # special case for layer 1 and 2
                if Hbot == 1
                    if Htop == 2
                        flux_1_Lc = t_length_Lc/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]*sticking(TerraceLength[Htop-1]))
                    end
                end
            
                # find length needed to reach exact Lmax
                t_length_Lmax = TerraceLength[Hbot]

                #flux for sub step 1
                flux_1_Lmax =  t_length_Lmax/(TerraceLength[Hbot]*sticking(TerraceLength[Hbot]) + TerraceLength[Hbot+1]/2*sticking(TerraceLength[Hbot+1])*(1-Ls/(Ls+TerraceLength[Hbot+1])))
            
                # see which one happens first
                sub_trigger = 0
                if flux_1_Lc >= flux_1_Lmax
                    sub_trigger = 1 # Lmax correction first
                else
                    sub_trigger = 2 # Lc correction first
                end
                        
                if sub_trigger == 1 # Lmax correction first
                    F_temp = copy(F) # save the default flux
                    F = copy(flux_1_Lmax) # replace the flux with flux_1 for sub step 1

        
                    # recalculate the v_array for sub step Lmax
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_1[j] >= Lmax      
                            if L_step_1[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_1[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end

                    
                    #update the L array with sub step Lmax
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lc
                            L_step_1[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step Lmax
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lc
                            if j==Htop 
                                Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step Lc
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    L_step_2 = copy(L_step_1)
                
                    # Calculate the terrace lengths.
                    TerraceLength = zeros(Hmax)
                    for j in Hbot:Htop
                        TerraceLength[j] = L_step_2[j] - L_step_2[j+1]
                    end
                
                    # find length needed to reach exact Lc after Lmax correction
                    t_length_Lc_sub = Lc - TerraceLength[Htop]
            
                    # now find the flux need for Lc correction after Lmax correction
                    flux_1_Lc_sub =  t_length_Lc_sub/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]/2*sticking(TerraceLength[Htop-1])*(1+Ls/(Ls+TerraceLength[Htop-1])))
                    # special case for layer 1 and 2
                    if Hbot == 1
                        if Htop == 2
                            flux_1_Lc_sub = t_length_Lc_sub/(TerraceLength[Htop]*sticking(TerraceLength[Htop]) + TerraceLength[Htop-1]*sticking(TerraceLength[Htop-1]))
                        end
                    end
                
                    F = copy(flux_1_Lc_sub) # replace the flux with flux_1 for sub step 1

                    # recalculate the v_array for sub step 2
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_2[j] >= Lmax      
                            if L_step_2[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_2[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end
            
                    
                    #update the L array with sub step Lc
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lc
                            L_step_2[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step Lc
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lc
                            if j==Htop 
                                Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step 3
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    L_step_3 = copy(L_step_2)

                
                    # Calculate the terrace lengths.
                    TerraceLength = zeros(Hmax)
                    for j in Hbot:Htop
                        TerraceLength[j] = L_step_3[j] - L_step_3[j+1]
                    end
                    
                    # finish the leftover
                    flux_2 = F_temp - flux_1_Lmax - flux_1_Lc_sub
                
                    F = copy(flux_2)
                
                    # recalculate the v_array for sub step 2
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_3[j] >= Lmax      
                            if L_step_3[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_3[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end
            
                    
                    #update the L array with sub step 3
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lc
                            L_step_3[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step 3
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lc
                            if j==Htop 
                                 Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step 3
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    F = copy(F_temp) # return the flux to the default
                    L_array = copy(L_step_3)
                end
                
                
                if sub_trigger == 2 # Lc correction first
                    F_temp = copy(F) # save the default flux
                    F = copy(flux_1_Lc) # replace the flux with flux_1 for sub step 1

        
                    # recalculate the v_array for sub step Lmax
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_1[j] >= Lmax      
                            if L_step_1[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_1[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end

                    
                    #update the L array with sub step Lc
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lc
                            L_step_1[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step Lc
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lc
                            if j==Htop 
                                Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step Lmax
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_1[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    L_step_2 = copy(L_step_1)
                
                    # Calculate the terrace lengths.
                    TerraceLength = zeros(Hmax)
                    for j in Hbot:Htop
                        TerraceLength[j] = L_step_2[j] - L_step_2[j+1]
                    end
                    
                    
                    # find length needed to reach exact Lmax after Lc correction
                    t_length_Lmax_sub = TerraceLength[Hbot]

                    # now find the flux need for Lmax correction after Lc correction
                    flux_1_Lmax_sub =  t_length_Lmax_sub/(TerraceLength[Hbot]*sticking(TerraceLength[Hbot]) + TerraceLength[Hbot+1]/2*sticking(TerraceLength[Hbot+1])*(1-Ls/(Ls+TerraceLength[Hbot+1])))
                
                
                    F = copy(flux_1_Lmax_sub) # replace the flux with flux_1 for sub step 1

                    # recalculate the v_array for sub step 2
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_2[j] >= Lmax      
                            if L_step_2[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_2[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end
            
                    
                    #update the L array with sub step Lmax
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lc
                            L_step_2[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub step Lmax
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lc
                            if j==Htop 
                                Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step 3
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_2[j] >= Lmax
                            newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    L_step_3 = copy(L_step_2)

                    # Calculate the terrace lengths.
                    TerraceLength = zeros(Hmax)
                    for j in Hbot:Htop
                        TerraceLength[j] = L_step_3[j] - L_step_3[j+1]
                    end
                    
                    # finish the leftover
                    flux_2 = F_temp - flux_1_Lmax_sub - flux_1_Lc
                
                    F = copy(flux_2)
                    # recalculate the v_array for sub step 2
                    v = zeros(Hmax)
                    for j in Hbot:Htop-1
                        if L_step_3[j] >= Lmax      
                            if L_step_3[j+1] < Lc
                                v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,true)
                
                            else
                                v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,true)     
                            end
            
                        elseif L_step_3[j+1] < Lc
                            v[j] =  f(TerraceLength[j+1],Ls,true) + g(TerraceLength[j],Ls,false) 
                        else
                            v[j] =  f(TerraceLength[j+1],Ls,false) + g(TerraceLength[j],Ls,false) 
                        end
                    end
            
                    
                    #update the L array with sub 3
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lc
                            L_step_3[j+1] += v[j]
                        end
                    end
            
            
                    # find correct Htop after sub 3
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lc
                            if j==Htop 
                                 Htop+=1            
                            end
                        end
                    end
        
        
                    # find correct Hbot for sub step 3
                    newHbot = 1
                    for j in Hbot:Htop
                        if L_step_3[j] >= Lmax
                        newHbot = j
                        end
                    end
                    Hbot = newHbot
        
                    F = copy(F_temp) # return the flux to the default
                    L_array = copy(L_step_3)
                end
            end
        end

    
    L_1 =  copy(L_array)

    terrace = []
    for k in 1:length(L_1)-1
        terrace = append!(terrace, L_array[k]-L_array[k+1])
    end    
       
    mean = sum(terrace.*(1:length(terrace)))/sum(terrace)
    rms = sqrt(sum(terrace.*((1-mean:length(terrace)-mean).^2))/sum(terrace))
            
    rms_array[i] = rms
    avg_h_array[i] = mean 
    end
return rms_array, avg_h_array

end

end # end module

Main.zeno

In [2]:
# using Pkg
# Pkg.add("ProgressBars")

In [3]:
using Statistics;using PyPlot; pygui(true)

true

In [4]:
0.005*3000000

15000.0

In [5]:
a = 1
Hmax = 16000
F = 0.005
Lc = 0.5
Ls = 1000
tmax = 3000000
Lmax = 20
Ld = 0.1
B = 0.2

rms, avg_h = zeno.zenogrowth(Lmax, Hmax, tmax; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)

0.0%┣                                      ┫ 0/3.0M [00:01<-569:-51:-25, -1s/it]
1.1%┣▍                                    ┫ 34.0k/3.0M [01:01<01:28:28, 559it/s]
2.2%┣▉                                    ┫ 67.4k/3.0M [02:01<01:27:37, 558it/s]
3.3%┣█▎                                   ┫ 99.8k/3.0M [03:01<01:27:31, 552it/s]
4.4%┣█▋                                  ┫ 131.6k/3.0M [04:01<01:27:28, 547it/s]
5.4%┣██                                  ┫ 162.4k/3.0M [05:01<01:27:34, 540it/s]
6.4%┣██▎                                 ┫ 192.7k/3.0M [06:01<01:27:36, 534it/s]
7.4%┣██▋                                 ┫ 222.2k/3.0M [07:01<01:27:41, 528it/s]
8.4%┣███                                 ┫ 250.9k/3.0M [08:01<01:27:48, 522it/s]
9.3%┣███▍                                ┫ 278.7k/3.0M [09:01<01:28:00, 515it/s]
10.2%┣███▋                               ┫ 306.6k/3.0M [10:01<01:27:58, 510it/s]
11.1%┣████                               ┫ 333.7k/3.0M [11:01<01:28:00, 505it/s]
12.0%┣████▏                 

62.6%┣█████████████████████▎            ┫ 1.9M/3.0M [01:42:01<01:00:58, 307it/s]
63.0%┣█████████████████████▍            ┫ 1.9M/3.0M [01:43:01<01:00:32, 306it/s]
63.4%┣█████████████████████▌            ┫ 1.9M/3.0M [01:44:01<01:00:06, 305it/s]
63.8%┣███████████████████████▋             ┫ 1.9M/3.0M [01:45:01<59:39, 304it/s]
64.2%┣███████████████████████▊             ┫ 1.9M/3.0M [01:46:01<59:13, 303it/s]
64.5%┣███████████████████████▉             ┫ 1.9M/3.0M [01:47:01<58:47, 302it/s]
64.9%┣████████████████████████             ┫ 1.9M/3.0M [01:48:01<58:21, 301it/s]
65.3%┣████████████████████████▏            ┫ 2.0M/3.0M [01:49:01<57:55, 300it/s]
65.7%┣████████████████████████▎            ┫ 2.0M/3.0M [01:50:01<57:29, 298it/s]
66.1%┣████████████████████████▍            ┫ 2.0M/3.0M [01:51:01<57:03, 298it/s]
66.4%┣████████████████████████▋            ┫ 2.0M/3.0M [01:52:01<56:36, 297it/s]
66.8%┣████████████████████████▊            ┫ 2.0M/3.0M [01:53:01<56:10, 296it/s]
67.2%┣██████████████████████

96.1%┣███████████████████████████████████▋ ┫ 2.9M/3.0M [03:24:01<08:17, 236it/s]
96.4%┣███████████████████████████████████▋ ┫ 2.9M/3.0M [03:25:01<07:42, 235it/s]
96.7%┣███████████████████████████████████▊ ┫ 2.9M/3.0M [03:26:01<07:08, 235it/s]
96.9%┣███████████████████████████████████▉ ┫ 2.9M/3.0M [03:27:01<06:34, 234it/s]
97.2%┣████████████████████████████████████ ┫ 2.9M/3.0M [03:28:01<05:59, 234it/s]
97.5%┣████████████████████████████████████ ┫ 2.9M/3.0M [03:29:01<05:24, 233it/s]
97.8%┣████████████████████████████████████▏┫ 2.9M/3.0M [03:30:01<04:49, 233it/s]
98.0%┣████████████████████████████████████▎┫ 2.9M/3.0M [03:31:01<04:15, 232it/s]
98.3%┣████████████████████████████████████▍┫ 2.9M/3.0M [03:32:01<03:40, 232it/s]
98.6%┣████████████████████████████████████▌┫ 3.0M/3.0M [03:33:01<03:05, 231it/s]
98.8%┣████████████████████████████████████▋┫ 3.0M/3.0M [03:34:01<02:31, 231it/s]
99.1%┣████████████████████████████████████▊┫ 3.0M/3.0M [03:35:01<01:55, 230it/s]
99.4%┣██████████████████████

([0.0631968827902374, 0.08921152818225564, 0.10905529131329306, 0.12568291093612755, 0.14024171595342705, 0.15332149441181236, 0.16577602410500003, 0.17747534888010807, 0.18844934864742566, 0.19881765084087302  …  70.71422648646531, 70.71422735627175, 70.71422827727257, 70.71422924774053, 70.7142302659793, 70.71423133032279, 70.71423243913488, 70.71423359080862, 70.71423478376582, 70.71423601645634], [1.0040099254968937, 1.0080230663543397, 1.0120379692675128, 1.0160539225299772, 1.0200705665345806, 1.0240876978356952, 1.0281052445441927, 1.0321230893277458, 1.0361412652858384, 1.040159748519301  …  14283.44798960845, 14283.45278413855, 14283.457578704165, 14283.46237330424, 14283.46716793778, 14283.471962603746, 14283.476757301143, 14283.481552029021, 14283.486346786416, 14283.491141572416])

In [6]:
figure()

# plot(avg_h,rms,"-o")
plot(log.(avg_h),log.(rms),"-o")

plot(log.(avg_h),log.(sqrt.(avg_h)),"-")


plt.tight_layout()

In [7]:
using DelimitedFiles
using CSV


# writedlm("avg_h_3p5_ls10_b02_15k.csv",  avg_h)
# writedlm("rms_3p5_ls10_b02_15k.csv",  rms)

In [ ]:
a = 1
Hmax = 1200
F = 0.005
Lc = 0.5
Ls = 0.015
tmax = 200000
Lmax = 6
Ld = 0.1
B = 0.00

rms, avg_h = zeno.zenogrowth(Lmax, Hmax, tmax; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)

In [ ]:
figure()

plot(avg_h,rms,"-o")

plt.tight_layout()

In [ ]:
# writedlm("avg_h_low_ls.csv",  avg_h)
# writedlm("rms_low_ls.csv",  rms)

In [ ]:
Lmax_array = vcat(1:0.5:2.5,2.6:0.2:3.4,3.5:0.05:4.8,5.0:0.2:6,7:10,10:1:30)
Lmax_array = vcat(1:0.5:2.5,2.6:0.2:3.4,3.5:0.05:5.0,5.2:0.2:7,8:1:30,32:2:50)
Lmax_array = vcat(1:0.5:3,3.2:0.1:3.5,3.55:0.05:5.2,5.4:0.2:8,9:1:30,32:2:50)

# Lmax_array = 10:1:30
println(Lmax_array)

In [ ]:
a = 1
Hmax = 1800
F = 0.005
Lc = 0.5
Ls = 0.015
tmax = 300000
# Lmax = 5
Ld = 0.1
B = 0.00



Lmax_array = vcat(1:1:10)


# figure()
rough = []

for i in 1:length(Lmax_array)
    
    rms, avg_h = zeno.zenogrowth(Lmax_array[i], Hmax, tmax; F, a=1.0, Ls=Ls, Lc=Lc, Ld= Ld, B=B)
    
#     plot(h_f,rms_f,"-o",label="Lmax = $(Lmax_array[i])")
    
    t_layer = (1/F)

    rms_ini = rms[ceil(Int,(tmax - 520*t_layer)):ceil(Int,(tmax - 500*t_layer))]


    rms_final = rms[ceil(Int,(tmax - 20*t_layer)):tmax]


    a = mean(rms_ini)
    b = mean(rms_final)
    c = (b-a)/a
    
#     println(Lmax_array[i])
#     println("mean:",a)
#     println("mean:",b)
#     println("diff:",c*100," %")
    println(c*100)
#     println("_")
    rough = append!(rough,c*100)  
    
end

# plt.plot(1:1500,sqrt.(1:1500),color="black",lw=3)

# tick_params(labelsize=15)
# xlabel("Thickness (layers)",fontsize=15)
# ylabel("RMS roughness (layers)",fontsize=15)


# legend(fontsize=15)

# tick_params(labelsize=15)
# tight_layout()

In [ ]:
pygui(false)

figure()

plot(Lmax_array,rough,"-o")

tight_layout()

In [ ]:
1/8/Ls+0.5

In [ ]:
sqrt(1/Ls)